In [1]:
pip install fastapi[all] scikit-learn uvicorn pandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import joblib
import pandas as pd

In [4]:
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(
    iris.data, iris.target, test_size=0.2, random_state=42
)

In [5]:
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [6]:
joblib.dump(clf, "iris_model.joblib")

['iris_model.joblib']

In [7]:
accuracy = clf.score(X_test, y_test)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 1.00


In [8]:
from fastapi import FastAPI
from pydantic import BaseModel
import joblib

app = FastAPI()
model = joblib.load("iris_model.joblib")
iris_target_names = load_iris().target_names

In [9]:
class IrisRequest(BaseModel):
    sepal_length: float
    sepal_width: float
    petal_length: float
    petal_width: float

In [10]:
@app.post("/predict")
def predict_species(request: IrisRequest):
    data = [[
        request.sepal_length,
        request.sepal_width,
        request.petal_length,
        request.petal_width
    ]]
    pred = model.predict(data)[0]
    species = iris_target_names[pred]
    return {"species": species}

In [ ]:
import nest_asyncio
import uvicorn

nest_asyncio.apply()

uvicorn.run(app, host="0.0.0.0", port=8000)

INFO:     Started server process [17892]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:63296 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:63296 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:63295 - "POST /predict HTTP/1.1" 200 OK
